In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
learning_rate = 0.001
dimension = 40

In [3]:
def data_preprocess(data_path:str)->pd.DataFrame:
    """
    数据预处理
    :param data_path: 数据路径
    :return: 处理后的数据
    """
    data_raw = pd.read_csv(data_path)   # 读取数据
    data_raw = data_raw.iloc[:, :3] # 去掉时间戳
    data_raw = data_raw.sample(frac=1, random_state=42) # 打乱数据
    return data_raw

In [4]:
data_raw = data_preprocess("ratings.csv")
data_raw.head()

,userId,movieId,rating
67037,432,77866,4.5
42175,288,474,3.0
93850,599,4351,3.0
6187,42,2987,4.0
12229,75,1610,4.0


In [5]:
# 划分k折
def k_fold_split(data:pd.DataFrame, k:int)->list:
    """
    划分k折
    :param data: 数据
    :param k: k折
    :return: k折数据
    """
    data_len = len(data)
    for i in range(k):
        start = int(i * data_len / k)
        end = int((i + 1) * data_len / k)
        # 返回训练集和验证集
        yield pd.concat([data.iloc[:start], data.iloc[end:]]), data.iloc[start:end]

In [6]:
for _train_set, _valid_set in k_fold_split(data_raw, 10):
    # print(len(_train_set), len(_valid_set))
    train_set = _train_set.copy()
    valid_set = _valid_set.copy()
    break

len(train_set), len(valid_set)

(90753, 10083)

In [7]:
class Node:
    """节点"""
    def __init__(self, dimension:int=2):
        """
        :param dimension: 节点维度
        """
        self.position = np.random.rand(dimension) # 随机初始化位置

In [8]:
def get_nodes(data:pd.DataFrame, dimension:int=40)->tuple[dict, dict]:
    """
    获取节点
    :param data: 数据
    :param dimension: 节点维度
    
    """
    user_nodes, item_nodes = {}, {}
    for user, _ in data.groupby("userId"):
        user_nodes[user] = Node(dimension)
    for item, _ in data.groupby("movieId"):
        item_nodes[item] = Node(dimension)
    return user_nodes, item_nodes

In [9]:
_user_nodes, _item_nodes = get_nodes(data_raw, dimension)

In [10]:
_user_nodes[1].__dict__

{'position': array([0.34424673, 0.13206273, 0.38045109, 0.36987059, 0.55081594,
        0.58829371, 0.99976971, 0.69078051, 0.73392781, 0.7991075 ,
        0.66362605, 0.06905537, 0.79766906, 0.27339348, 0.24495821,
        0.66270197, 0.89189692, 0.62026084, 0.71658974, 0.63534646,
        0.86368741, 0.43934154, 0.6943998 , 0.24257093, 0.56755608,
        0.72485404, 0.26078029, 0.17669406, 0.93296944, 0.99779346,
        0.33427413, 0.66137018, 0.57446261, 0.742707  , 0.4022649 ,
        0.45228668, 0.26555457, 0.41934779, 0.99063883, 0.76124717])}

In [11]:
def predict(valid_set:pd.DataFrame, user_nodes:dict, item_nodes:dict, maxR:float=5., minR:float=.5):
    """
    预测评分
    
    """
    # 为验证集添加一列，存储预测评分
    valid_set["predict"] = 0
    for i, row in valid_set.iterrows():
        user, item, rating = int(row["userId"]), int(row["movieId"]), row["rating"]
        user_node, item_node = user_nodes[user], item_nodes[item]
        # 计算欧氏距离
        pred_distance = np.linalg.norm(user_node.position - item_node.position)
        pred_rating = maxR - (maxR - minR) * pred_distance / 100
        if pred_rating > maxR:
            pred_rating = maxR
        elif pred_rating < minR:
            pred_rating = minR
        valid_set.loc[i, "predict"] = pred_rating
    return valid_set

In [12]:
def update(train_set:pd.DataFrame, user_nodes:dict, item_nodes:dict, learning_rate:float=0.001, maxR:float=5., minR:float=0.5)->None:
    """
    训练
    :param train_set: 训练集
    :param user_nodes: 用户节点
    :param item_nodes: 物品节点
    :param learning_rate: 学习率
    :param maxR: 最大评分
    :param minR: 最小评分
    """
    for _, row in train_set.iterrows():
        user, item, rating = int(row["userId"]), int(row["movieId"]), row["rating"]
        user_node, item_node = user_nodes[user], item_nodes[item]

        # 计算欧氏距离
        pred_distance = np.linalg.norm(user_node.position - item_node.position)
        real_distance = 100 * (maxR - rating) / (maxR - minR)
        error = real_distance - pred_distance
        direction = user_node.position - item_node.position
        # 更新节点位置
        to_move = learning_rate * error * direction
        user_node.position += to_move
        item_node.position -= to_move
        
    return user_nodes, item_nodes

In [13]:
def train(train_set:pd.DataFrame, user_nodes:dict, item_nodes:dict, epoch:int=20, learning_rate:float=0.001, maxR:float=5., minR:float=0.5):
    for i in range(epoch):
        user_nodes, item_nodes = update(train_set, user_nodes, item_nodes, learning_rate, maxR, minR)
    return user_nodes, item_nodes

In [14]:
from copy import deepcopy
_user_nodes_trained, _item_nodes_trained = train(train_set, deepcopy(_user_nodes), deepcopy(_item_nodes), epoch=20, learning_rate=learning_rate, maxR=5., minR=0.5)

In [15]:
_predictions = predict(valid_set, _user_nodes_trained, _item_nodes_trained, maxR=5., minR=0.5)

In [16]:
_predictions

,userId,movieId,rating,predict
67037,432,77866,4.5,3.474787
42175,288,474,3.0,3.215560
93850,599,4351,3.0,2.654645
6187,42,2987,4.0,3.575485
12229,75,1610,4.0,3.410093
...,...,...,...,...
84545,543,1387,5.0,3.518706
52065,339,1580,2.0,3.611221
92269,597,1090,5.0,4.274593
18346,116,30749,4.5,3.336013


In [17]:
mae = np.mean(np.abs(_predictions["predict"] - _predictions["rating"]))
rmse = np.sqrt(np.mean(np.square(_predictions["predict"] - _predictions["rating"])))
print(f"MAE: {mae}, RMSE: {rmse}")

MAE: 0.6864917774214186, RMSE: 0.8768384558789274


In [18]:
def correction_user(train_set:pd.DataFrame, user_node:Node, item_nodes:dict[int, Node], maxR:float=5.0, minR:float=0.5):
    """
    修正用户位置
    :param train_set: 训练集
    :param user_node: 用户节点
    :param item_nodes: 电影节点
    """
    weight = {}
    for _, row in train_set.iterrows():
        movie_id = int(row["movieId"])
        item_node = item_nodes[movie_id]
        weight[movie_id] = 1 / np.linalg.norm(user_node.position - item_node.position)
    sim_sum = sum(weight.values())
    for movie_id in weight.keys():
        weight[movie_id] /= sim_sum

    correction = 0
    for _, row in train_set.iterrows():
        movie_id = int(row["movieId"])
        item_node = item_nodes[movie_id]
        real_rating = row["rating"]
        pred_distance = np.linalg.norm(user_node.position - item_node.position)
        pred_rating = maxR - (maxR - minR) * pred_distance / 100
        error = real_rating - pred_rating
        correction += error * weight[movie_id] 
    return correction

In [19]:
def correction_item(train_set:pd.DataFrame, item_node:Node, user_nodes:dict[int, Node], maxR:float=5.0, minR:float=0.5):
    """
    修正物品位置
    :param train_set: 训练集
    :param item_node: 物品节点
    :param user_nodes: 用户节点
    """
    weight = {}
    for _, row in train_set.iterrows():
        user_id = int(row["userId"])
        user_node = user_nodes[user_id]
        weight[user_id] = 1 / np.linalg.norm(user_node.position - item_node.position)
    sim_sum = sum(weight.values())
    for user_id in weight.keys():
        weight[user_id] /= sim_sum

    correction = 0
    for _, row in train_set.iterrows():
        user_id = int(row["userId"])
        user_node = user_nodes[user_id]
        real_rating = row["rating"]
        pred_distance = np.linalg.norm(user_node.position - item_node.position)
        pred_rating = maxR - (maxR - minR) * pred_distance / 100
        error = real_rating - pred_rating
        correction += error * weight[user_id]
    return correction

In [20]:
def get_correction(train_set:pd.DataFrame, user_nodes:dict[int, Node], item_nodes:dict[int, Node], maxR:float=5.0, minR:float=0.5):
    """
    获取修正值
    :param train_set: 训练集
    :param user_nodes: 用户节点
    :param item_nodes: 物品节点
    :return: 修正值
    """
    user_correction, item_correction = {}, {}
    for user, group in train_set.groupby("userId"):
        user_correction[user] = correction_user(group, user_nodes[user], item_nodes, maxR, minR)
    for item, group in train_set.groupby("movieId"):
        item_correction[item] = correction_item(group, item_nodes[item], user_nodes, maxR, minR)
    return user_correction, item_correction

In [21]:
_user_correction, _item_correction = get_correction(train_set, _user_nodes_trained, _item_nodes_trained, maxR=5., minR=0.5)

In [22]:
_user_correction

{1: 0.389468869760891,
 2: 0.17804396405812398,
 3: 0.7025513583782321,
 4: 0.45940167951949895,
 5: 0.1987104781325379,
 6: 0.22101196832559697,
 7: 0.30820430263452153,
 8: 0.1577081362000517,
 9: 0.3676099525494847,
 10: 0.40249240323773017,
 11: 0.2472333055452766,
 12: 0.7420314887133259,
 13: 0.28481860431394596,
 14: 0.17636525345780707,
 15: 0.2716638491324342,
 16: 0.06485981178615766,
 17: 0.07010382431751377,
 18: 0.15229518091781297,
 19: 0.01657846786854239,
 20: 0.32278611682351477,
 21: 0.17406873126655484,
 22: 0.028572721752578682,
 23: 0.1018615214875194,
 24: 0.05535942053491594,
 25: 0.5057535476989109,
 26: 0.055850322609580526,
 27: 0.28648314315017925,
 28: 0.04845176878809859,
 29: 0.14531224417777766,
 30: 0.4021681138147169,
 31: 0.2709551706029304,
 32: 0.13349503956574438,
 33: 0.2648060403611487,
 34: 0.3400624430061456,
 35: 0.38077626913870194,
 36: 0.06925044266998498,
 37: 0.35995172832173633,
 38: 0.1930540031457667,
 39: 0.23836173460938656,
 40: 0.30

In [23]:
_item_correction

{1: 0.21886403257280232,
 2: 0.16509293504959968,
 3: 0.2164060020458355,
 4: 0.09834730015866486,
 5: 0.18325919737830249,
 6: 0.18659177677941546,
 7: 0.1418723777717872,
 8: 0.25587428054149197,
 9: 0.1713537976533951,
 10: 0.14792259574801897,
 11: 0.16536877973258218,
 12: 0.21272082191866293,
 13: 0.03348137420301327,
 14: 0.17016113021993162,
 15: 0.25022188929166483,
 16: 0.2573427624718424,
 17: 0.3277924321518598,
 18: 0.20339824931452483,
 19: 0.08346278136583486,
 20: 0.11804684246352004,
 21: 0.13995694678293097,
 22: 0.09577032031491681,
 23: 0.11854224027082187,
 24: 0.21351842957243966,
 25: 0.30051316620932766,
 26: -0.010976462964630458,
 27: 0.3123137231696957,
 28: 0.4564075741015733,
 29: 0.28430992577481407,
 30: 0.38784155631081196,
 31: 0.19002899392063602,
 32: 0.2101979987427871,
 34: 0.25819737362397815,
 36: 0.16368008607228324,
 38: -0.07933766836075606,
 39: 0.15782956810915807,
 40: 0.6651192260165245,
 41: 0.13604950857295264,
 42: 0.061239129205672926,


In [24]:
def DTEC(predictions:pd.DataFrame, user_corrections:dict, item_corrections:dict):
    """
    DTEC
    :param predictions: 预测评分
    :param user_corrections: 用户修正值
    :param item_corrections: 物品修正值
    """
    for i, row in predictions.iterrows():
        user, item = int(row["userId"]), int(row["movieId"])
        if user in user_corrections.keys() and item in item_corrections.keys():
            predictions.loc[i, "predict"] += (user_corrections[user] + item_corrections[item]) / 2
    return predictions

In [25]:
predictions_corrected = DTEC(_predictions, _user_correction, _item_correction)

In [26]:
predictions_corrected

,userId,movieId,rating,predict
67037,432,77866,4.5,3.510421
42175,288,474,3.0,3.408314
93850,599,4351,3.0,2.725577
6187,42,2987,4.0,3.761484
12229,75,1610,4.0,3.574382
...,...,...,...,...
84545,543,1387,5.0,4.079841
52065,339,1580,2.0,3.764249
92269,597,1090,5.0,4.544785
18346,116,30749,4.5,3.473921


In [27]:
# 计算MAE和RMSE
mae = np.abs(predictions_corrected["predict"] - predictions_corrected["rating"]).mean()
rmse = np.sqrt(((predictions_corrected["predict"] - predictions_corrected["rating"]) ** 2).mean())
print("mae:", mae, "rmse:", rmse)

mae: 0.6561959517161507 rmse: 0.8603222084326587


In [28]:
def SCoR(rating_path:str, fold:int=10, learning_rate:float=0.001, dimension:int=40, epoch:int=20):
    """
    SCoR
    :param rating_path: 评分路径
    :param fold: k折验证折数
    :param learning_rate: 学习率
    :param dimension: 节点维度
    :param epoch: 迭代次数
    """
    data = data_preprocess(rating_path)
    maxR, minR = data["rating"].max(), data["rating"].min() # 最大评分和最小评分
    
    MAEs, RMSEs = [], []
    for i, (train_set, valid_set) in enumerate(k_fold_split(data, fold)):
        print("fold:", i+1)
        # 转为节点
        user_nodes, item_nodes = get_nodes(data, dimension=dimension)
        # 训练
        user_nodes_trained, item_nodes_trained = train(train_set, user_nodes, item_nodes, epoch=epoch, learning_rate=learning_rate, maxR=maxR, minR=minR)
        # 预测 
        predictions = predict(valid_set.copy(), user_nodes_trained, item_nodes_trained, maxR, minR)
        
        mae = np.abs(predictions["predict"] - predictions["rating"]).mean()
        rmse = np.sqrt(((predictions["predict"] - predictions["rating"]) ** 2).mean())
        MAEs.append(mae)
        RMSEs.append(rmse)
        print("MAE:", mae, "RMSE:", rmse)
    print()
    print("MAE:", np.mean(MAEs), "RMSE:", np.mean(RMSEs))

In [29]:
SCoR("ratings.csv")

fold: 1
MAE: 0.6862689516248064 RMSE: 0.8766695943702542
fold: 2
MAE: 0.6780990766817537 RMSE: 0.8656463064023623
fold: 3
MAE: 0.6714204519325908 RMSE: 0.8571250117639743
fold: 4
MAE: 0.6761497453452642 RMSE: 0.8641452073650268
fold: 5
MAE: 0.6805942007262477 RMSE: 0.8646033382005867
fold: 6
MAE: 0.6615183418706417 RMSE: 0.8482424408601811
fold: 7
MAE: 0.6804845021948958 RMSE: 0.8701335207473528
fold: 8
MAE: 0.6726799146502166 RMSE: 0.8566891228390606
fold: 9
MAE: 0.6774110398967629 RMSE: 0.8648173917287734
fold: 10
MAE: 0.6824952692588625 RMSE: 0.8739535604653419

MAE: 0.6767121494182041 RMSE: 0.8642025494742913


In [30]:
def DTEC_SCoR(rating_path:str, fold:int=10, learning_rate:float=0.001, dimension:int=40, epoch:int=20):
    """
    SCoR
    :param rating_path: 评分路径
    :param fold: k折验证折数
    :param learning_rate: 学习率
    :param dimension: 节点维度
    :param epoch: 迭代次数
    """
    data = data_preprocess(rating_path)
    maxR, minR = data["rating"].max(), data["rating"].min() # 最大评分和最小评分
    
    MAEs, RMSEs = [], []
    for i, (train_set, valid_set) in enumerate(k_fold_split(data, fold)):
        print("fold:", i+1)
        # 转为节点
        user_nodes, item_nodes = get_nodes(data, dimension=dimension)
        # 训练
        user_nodes_trained, item_nodes_trained = train(train_set, user_nodes, item_nodes, epoch=epoch, learning_rate=learning_rate, maxR=maxR, minR=minR)
        # 预测 
        predictions = predict(valid_set.copy(), user_nodes_trained, item_nodes_trained, maxR, minR)
        # 修正
        user_correction, item_correction = get_correction(train_set, user_nodes_trained, item_nodes_trained, maxR, minR)
        predictions_corrected = DTEC(predictions, user_correction, item_correction)
        
        mae = np.abs(predictions_corrected["predict"] - predictions_corrected["rating"]).mean()
        rmse = np.sqrt(((predictions_corrected["predict"] - predictions_corrected["rating"]) ** 2).mean())
        MAEs.append(mae)
        RMSEs.append(rmse)
        print("MAE:", mae, "RMSE:", rmse)
    print()
    print("MAE:", np.mean(MAEs), "RMSE:", np.mean(RMSEs))

In [31]:
DTEC_SCoR("ratings.csv")

fold: 1
MAE: 0.6559381182767131 RMSE: 0.8596331774050476
fold: 2
MAE: 0.6443721431096946 RMSE: 0.8449441055600881
fold: 3
MAE: 0.6385980906053179 RMSE: 0.8362012770978847
fold: 4
MAE: 0.6445486202411822 RMSE: 0.8450446987699096
fold: 5
MAE: 0.6478037416644851 RMSE: 0.8435379415575572
fold: 6
MAE: 0.6336624763104698 RMSE: 0.8313378626455386
fold: 7
MAE: 0.6526870486344993 RMSE: 0.8530468851427653
fold: 8
MAE: 0.6417581547644071 RMSE: 0.8350012866345615
fold: 9
MAE: 0.6468259303588679 RMSE: 0.8441576078622741
fold: 10
MAE: 0.653491392822005 RMSE: 0.8557411808421733

MAE: 0.6459685716787642 RMSE: 0.84486460235178


In [63]:
_user_nodes[1].__dict__

{'position': array([0.37204339, 0.54260116, 0.64842674, 0.85786069, 0.85901051,
        0.1888858 , 0.47824869, 0.92907306, 0.59637834, 0.67971148,
        0.64871852, 0.69236069, 0.44102094, 0.46730699, 0.28271299,
        0.6140472 , 0.97938074, 0.49360796, 0.71938493, 0.93675535,
        0.21631327, 0.07883136, 0.01722285, 0.53283554, 0.4459455 ,
        0.93785714, 0.92014504, 0.84016662, 0.3344173 , 0.38778218,
        0.23242349, 0.13706158, 0.9265114 , 0.8514605 , 0.97751015,
        0.27086168, 0.00531149, 0.77519219, 0.30120287, 0.64467569])}

In [64]:
_user_nodes_trained[1].__dict__

{'position': array([-2.38629497, -2.33890806,  1.27087751, -2.01780229, -0.41142093,
        -0.7750275 ,  0.78613277,  1.22055309,  2.30891667,  0.67344514,
         6.04984052,  3.02787452,  3.28042864,  1.13235687, -0.39424413,
         1.42007829,  1.46730391,  0.07171295,  2.33532712,  0.46166732,
         5.88551083, -1.92400719,  3.16403301,  0.05237008,  0.31012675,
        -0.02723566,  3.05032521,  0.74569589,  1.49936705, -0.20897518,
         4.43008162, -3.35135688, -3.04500404, -0.195118  ,  1.10259608,
         3.2419302 , -1.71742275, -2.08107767, -1.12574462, -1.92040704])}

In [65]:
def get_weight(train_set:pd.DataFrame, user_nodes:dict, item_nodes:dict, maxR:float=5., minR:float=0.5):
    train_set_weighted = train_set.copy()
    train_set_weighted["weight"] = 0
    for user, group in train_set.groupby("userId"):
        print(user)
        user_node = user_nodes[user]
        mse = 0
        for _, row in group.iterrows():
            item, rating = int(row["movieId"]), row["rating"]
            item_node = item_nodes[item]
            pred_distance = np.linalg.norm(user_node.position - item_node.position)
            pred_rating = maxR - (maxR - minR) * pred_distance / 100
            mse += (pred_rating - rating) ** 2
        mse /= len(group)
        print(mse)
        for index, row in group.iterrows():
            item, rating = int(row["movieId"]), row["rating"]
            item_node = item_nodes[item]
            desire_distance = np.linalg.norm(user_node.position - item_node.position)
            actual_distance = 100 * (maxR - rating) / (maxR - minR)
            weight = np.exp(-0.2 * mse) * ((desire_distance - actual_distance) ** 2)
            train_set_weighted.loc[index, "weight"] = weight
    train_set_weighted["weight"] /= train_set_weighted["weight"].sum()
    return train_set_weighted

In [66]:
train_set_weighted = get_weight(deepcopy(train_set), _user_nodes_trained, _item_nodes_trained)

1
0.6698195659115781
2
0.4081107261775119
3
1.3649660480395458
4
1.1673409918227093
5
0.5844010864743189
6
0.4032241782555559
7
0.9363022750283693
8
0.564374197336013
9
0.526183136834738
10
0.8557589243549197
11
0.38196423070879054
12
0.9054845270061769
13
0.43254776587414717
14
0.6520607598086733
15
0.8096941699853875
16
0.20410521211812002
17
0.19685570114219417
18
0.1837032698224068
19
0.3248506717001031
20
0.6331783465506315
21
0.47531132920789043
22
1.395149356062516
23
0.26224608309789543
24
0.13306953759351473
25
0.3016436675438116
26
0.1574887472843549
27
0.6374301571913727
28
0.31259577356455326
29
0.21112102398684834
30
0.38674243214816995
31
0.672380857461019
32
0.38762873779616047
33
0.604604790250168
34
1.2374768074852212
35
0.5156603045445881
36
0.4824989773815218
37
0.6144193658960759
38
0.669574897961307
39
0.6855314677961613
40
0.7114809043188409
41
0.799856572117184
42
0.6579205534451281
43
1.1594589853247104
44
0.6276224069792534
45
0.6062796952906487
46
0.4816023271

In [67]:
train_set_weighted

,userId,movieId,rating,weight
76059,479,351,2.0,1.122176e-05
14430,91,2431,2.0,2.278500e-05
43498,292,111,2.0,2.263676e-05
73590,474,1784,3.5,3.257891e-08
19181,124,110,3.5,1.021290e-05
...,...,...,...,...
6265,42,4005,4.0,2.518267e-06
54886,364,141,4.0,2.098944e-06
76820,480,6867,4.0,2.377445e-06
860,6,981,3.0,5.031883e-06


In [68]:
def update_weighted(train_set_weighted:pd.DataFrame, user_nodes:dict, item_nodes:dict, learning_rate:float=0.001, maxR:float=5., minR:float=0.5)->None:
    """
    训练
    :param train_set: 带权训练集
    :param user_nodes: 用户节点
    :param item_nodes: 物品节点
    :param learning_rate: 学习率
    :param maxR: 最大评分
    :param minR: 最小评分
    """
    # 随机选择n行
    train_set_weighted = train_set_weighted.sample(frac=1, weights=train_set_weighted["weight"])
    for _, row in train_set_weighted.iterrows():
        user, item, rating = int(row["userId"]), int(row["movieId"]), row["rating"]
        user_node, item_node = user_nodes[user], item_nodes[item]

        # 计算欧氏距离
        pred_distance = np.linalg.norm(user_node.position - item_node.position)
        real_distance = 100 * (maxR - rating) / (maxR - minR)
        error = real_distance - pred_distance
        direction = user_node.position - item_node.position
        # 更新节点位置
        to_move = learning_rate * error * direction
        user_node.position += to_move
        item_node.position -= to_move
    
    return user_nodes, item_nodes

In [69]:
def train_weighted(train_set_weighted:pd.DataFrame, user_nodes:dict, item_nodes:dict, epoch:int=20000, learning_rate:float=0.001, maxR:float=5., minR:float=0.5):
    predictions = None
    for i in range(epoch):
        user_nodes, item_nodes = update(train_set_weighted, user_nodes, item_nodes, learning_rate, maxR, minR)
        print(i)
        predictions = predict(valid_set, user_nodes, item_nodes, maxR, minR)
        mae = np.abs(predictions["predict"] - predictions["rating"]).mean()
        rmse = np.sqrt(((predictions["predict"] - predictions["rating"]) ** 2).mean())
        print("mae:", mae, "rmse:", rmse)
    return user_nodes, item_nodes, predictions

In [70]:
_user_nodes_trained_weighted, _item_nodes_trained_weighted, _predictions_weighted = train_weighted(train_set_weighted, deepcopy(_user_nodes_trained), deepcopy(_item_nodes_trained), epoch=20, learning_rate=learning_rate, maxR=5., minR=0.5)

0
mae: 0.6861201532064144 rmse: 0.8766251252628445
1
mae: 0.6863208943624184 rmse: 0.8768170719441207
2
mae: 0.6865271179062004 rmse: 0.8770240300763986
3
mae: 0.6867380778450589 rmse: 0.8772420467493333
4
mae: 0.6869448379404228 rmse: 0.8774679369840557
5
mae: 0.6871616157478762 rmse: 0.8776991173593623
6
mae: 0.687392184099465 rmse: 0.8779334804453673
7
mae: 0.6876207775165712 rmse: 0.8781692997778273
8
mae: 0.6878456757645988 rmse: 0.8784051575945719
9
mae: 0.6880658598480002 rmse: 0.8786398894446312
10
mae: 0.688273909337059 rmse: 0.8788725412383769
11
mae: 0.6884768823385026 rmse: 0.8791023354373144
12
mae: 0.6886806683097407 rmse: 0.8793286439551615
13
mae: 0.688887630021832 rmse: 0.8795509660091296
14
mae: 0.6890925184319834 rmse: 0.8797689096627327
15
mae: 0.6892900342469097 rmse: 0.8799821761726979
16
mae: 0.6894801047614899 rmse: 0.8801905465210447
17
mae: 0.6896631584591916 rmse: 0.8803938697029402
18
mae: 0.6898397282121105 rmse: 0.8805920524712644
19
mae: 0.690010801178861